<a href="https://colab.research.google.com/github/cvetyshayasiren/colabs/blob/main/Music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Код

In [8]:
#imports
from PIL import Image, ImageDraw, ImageFont
import ipywidgets as widgets
from random import choice
import requests, zipfile, io

In [9]:
url = 'http://fonts.google.com/download?family=Montserrat'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("Montserrat")

In [10]:
class Scale:
  def __init__(self):
    self.major = [0, 2, 4, 5, 7, 9, 11]
    self.minor = [0, 2, 3, 5, 7, 8, 10]
    self.ionian = [0, 2, 4, 5, 7, 9, 11]
    self.dorian = [0, 2, 3, 5, 7, 9, 10]
    self.phrygian = [0, 1, 3, 5, 7, 8, 10]
    self.lydian = [0, 2, 4, 6, 7, 9, 11]
    self.mixolydian = [0, 2, 4, 5, 7, 9, 10]
    self.aeolian = [0, 2, 3, 5, 7, 8, 10]
    self.locrian = [0, 1, 3, 5, 6, 8, 10]
    self.harmonic_major = [0, 2, 4, 5, 7, 8, 11]
    self.melodic_major = [0, 2, 4, 5, 7, 8, 10]
    self.harmonic_minor = [0, 2, 3, 5, 7, 8, 11]
    self.melodic_minor = [0, 2, 3, 5, 7, 9, 11]
    self.pentatonic = [0, 2, 4, 7, 9]
    self.chromatic = list(range(12))

In [11]:
class Color:
  def __init__(self):
    self.secondary = '#000000'
    self.primary = '#FFFFFF'
    self.accent_1 = '#64BE46'
    self.accent_2 = '#B22222'
    self.text = '#FF4500'
    self.random = lambda: '#%02x%02x%02x' % tuple(choice(range(1, 256)) for i in [0] * 3)

In [12]:
class Ui():
  '''интерфейс пользователя'''

  def __init__(self):
    self.scale = Scale()
    self.colors = Color()
    self.btn_go = widgets.Button(description='Показать 🎹', button_style='success')
    self.btn_go.on_click(self.btn_go_clicked)
    self.drop = widgets.Dropdown(options=['C', 'C#/Db', 'D', 'D#/Eb', 'E', 'F', 'F#/Gb', 'G', 'G#/Ab', 'A', 'A#/Hb', 'H'], 
                                 description='Тоника:', disabled=False)
    self.color = widgets.ColorPicker(concise=True, value=self.colors.accent_2, disabled=False)
    self.drop_2 = widgets.Dropdown(options=list(self.scale.__dict__), description='Звукоряд:', disabled=False)
    self.color_2 = widgets.ColorPicker(concise=True, value=self.colors.accent_1, disabled=False)
    self.pic = widgets.Image(format='png')
    self.size = widgets.SelectionSlider(options=list(range(1, 11)), value=5, description='Размер: ')
    self.rows = widgets.SelectionSlider(options=list(range(1, 4)), value=1, description='Октав: ')
    self.text = widgets.Textarea(value='lol lal lel', rows=1)
    self.piano = Piancture()
    self.show_pic()

  def display(self):
    hbox = widgets.HBox([self.drop, self.color, self.drop_2, self.color_2])
    hbox_2 = widgets.HBox([self.size, self.rows, self.btn_go])
    items = [hbox, self.pic, hbox_2]
    display(*items)

  def btn_go_clicked(self, b):
    self.colors.accent_1 = self.color_2.value
    self.colors.accent_2 = self.color.value
    self.piano.modulation(ton=self.drop.value, 
                          scale=self.scale.__dict__[self.drop_2.value], 
                          size=self.size.value, row = self.rows.value)
    self.show_pic()

  def show_pic(self):
    self.pic.set_value_from_file('keys.png')

In [13]:
class Piancture():
  '''Создание картинки с раскладкой пианино'''

  def __init__(self, ton = 'C', scale = Scale().major, size = 5, row = 1):
    
    self.colors = Color()
    self.ton = self.get_ton(ton)
    self.scale = scale
    self.size = self.set_size(size)
    self.row = row
    self.keys = self.draw_keys()

  def get_ton(self, ton):
    if type(ton) == str:
      ton = {'C': 0, 'C#/DB': 1, 'D': 2, 'D#/EB': 3, 
             'E': 4, 'F': 5, 'F#/GB': 6, 'G': 7, 
             'G#/AB': 8, 'A': 9, 'A#/HB': 10, 'H': 11}[ton.upper()]
    return ton

  def set_size(self, size = 5):
      size = list(zip([x * 112 for x in range(1, 12)], 
                      [y * 56 for y in range(1, 12)]))[size]
      return size

  def display(self):
    display(self.keys)

  def modulation(self, ton = None, scale = None, size = None, row = None):
    if ton: self.ton = self.get_ton(ton)
    if scale: self.scale = scale
    if size: self.size = self.set_size(size)
    if row: self.row = row
    self.keys = self.draw_keys()

  def draw_keys(self) -> Image:
    image = Image.new('RGB', self.size, self.colors.primary)
    draw = ImageDraw.Draw(image)
    w, h = self.size
    frame = []

    for id, x in enumerate(range(w//7, w, w//7)):
      draw.line((x, 0, x, h), fill=self.colors.secondary, width=1)
      frame.append([x - w//14, h - h//4])
      if id != 2:
        draw.rectangle((x - w//16, 0, x + w//16, h//2), fill=self.colors.secondary, outline=self.colors.primary, width=2)
        frame.append([x, h//4])

    frame.append([w - w//14, h - h//4])
    draw.rectangle((0, 0, w - 1, h - 1), outline=self.colors.secondary, width=2)
    tonality = [(self.ton + t) % 12 for t in self.scale]
    k = self.get_k()
    font = ImageFont.truetype("Montserrat/static/Montserrat-Regular.ttf", size=w//30)
    chord = iter(self.chords())
    step = iter(('I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII'))

    for t in tonality:
      fill = self.colors.accent_1 if t != tonality[0] else self.colors.accent_2
      ch = next(chord)
      text = f"{k[t]}{ch[0] if '/' not in k[t] else ''}\n{ch[2]}\n{next(step)} {ch[1]}".replace('\n\n', '\n')
      w, h = draw.textsize(text, font)
      f = frame[t]
      draw.multiline_text((f[0] - (w//2), f[1] - (h//2)), text, fill=fill, font=font, align='center')

    if self.row > 1:
      _image = Image.new('RGB', (self.size[0] * self.row, self.size[1]), self.colors.primary)
      x = 0
      for i in range(self.row):
        _image.paste(image, (x, 0))
        x += self.size[0]
      image = _image

    image.save("keys.png", quality=100)
    return image

  def get_k(self):
    if self.scale in ('major', 'minor'):
      pass
    k = {0: 'C', 1: 'C#/Db', 2: 'D', 3: 'D#/Eb', 
        4: 'E', 5: 'F', 6: 'F#/Gb', 7: 'G', 
        8: 'G#/Ab', 9: 'A', 10: 'A#/Hb', 11: 'H'}
    return k

  def chords(self):
    sc = self.scale + [s + 12 for s in self.scale]
    chords = []
    chords_dict = {
        '434': ['', 'Δ', '4|3|4'],
        '344': ['m', 'mΔ', '3|4|4'],
        '443': ['aug', '+M7', '4|4|3'],
        '433': ['', '7', '4|3|3'],
        '343': ['m', 'm7', '3|4|3'],
        '334': ['dim', 'mØ', '3|3|4'],
        '333': ['dim', '°7', '3|3|3'],
    }
    for id, i in enumerate(self.scale):
      tertia = sc[id+2] - i
      quinta = sc[id+4] - sc[id+2]
      septima = sc[id+6] - sc[id+4] if len(sc) > 10 else ''
      structure = f'{tertia}{quinta}{septima}'
      chords.append(chords_dict.get(structure, ['', '', '']))
    return chords

# Визуал

In [14]:
ui = Ui()
ui.display()

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02\xa0\x00\x00\x01P\x08\x02\x00\x00\x00\xef$[\x16\x…

# Тесты

In [ ]:
scale = [0, 2, 4, 5, 7, 9, 11]
ton = 2

def signs(scale, ton):
  key = {0: 'C', 1: 'C#/Db', 2: 'D', 3: 'D#/Eb', 4: 'E', 5: 'F', 
        6: 'F#/Gb', 7: 'G', 8: 'G#/Ab', 9: 'A', 10: 'A#/Hb', 11: 'H'}
  dies = {1: 'C#', 3: 'D#', 6: 'F#', 8: 'G#', 10: 'A#'}
  bemol = {1: 'Db', 3: 'Eb', 6: 'Gb', 8: 'Ab', 10: 'Hb'}

  scale = [(ton + t) % 12 for t in scale]
  out = []
  for id, i in enumerate(scale):
    if i in [1, 3, 6, 8, 10]:
      if (i - 1) % 12 in scale: 
        out.append(bemol[i])
        continue
      if (i + 1) % 12 in scale: 
        out.append(dies[i])
        continue
    out.append(key[i])
  return out

print(signs(scale, ton))


['D', 'E', 'F#', 'G', 'A', 'H', 'C#']
